In [1]:
from stir_lib import *
import numpy as np


from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [16]:
n = 5000
d = 10
alpha  = 0.1

n_corr  = int(alpha*n)

Idx= np.random.permutation(n)
corrIdx= Idx[0:n_corr]
cleanIdx=Idx[n_corr:n]

mu = 0
sigma = 1
X = np.random.normal(mu, sigma, (n, d))

w_star= np.random.normal(0,1, (d, 1))
w_star = w_star / np.linalg.norm(w_star)
w_adv= np.random.normal(0,1, (d, 1))
w_rand= np.random.normal(0,1, (d, 1))

y=np.zeros(shape=(n,1))
y[cleanIdx] = np.dot(X[cleanIdx,:], w_star)
y[corrIdx] = np.dot(X[corrIdx,:], w_adv)

In [17]:
#------------STIR-------------#
eta_range = np.linspace( 1.01, 3.01, 21 )
# STIR does not itself use alpha as a hyperparameter in the algorithm
# but does need it to perform cross-validation since the validation sets
# are also corrupted. To avoid an unfair comparison, We offer STIR a
# handicap by giving it the true value of alpha
alpha_range = np.linspace( alpha, alpha, 1 )

param_grid = dict( eta = eta_range, alpha = alpha_range )
cv = ShuffleSplit( n_splits = 5, test_size = 0.3, random_state = 42 )
grid = GridSearchCV( STIR( w_init = w_adv), param_grid=param_grid, cv = cv, refit = False )
grid.fit( X, y )

best = grid.best_params_

print("The best parameters for STIR are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))

The best parameters for STIR are {'alpha': 0.1, 'eta': 2.3099999999999996} with a score of -0.01


In [18]:
stir = STIR( eta = best["eta"], alpha = best["alpha"], M_init = np.power(10, 1), w_init = w_adv )
stir.fit( X, y, max_iter=400 )
print(np.linalg.norm(stir.w-w_star,2))

6.81602854186227e-16
